# Start - Datasets

In [ ]:
%%time

# Step1 - Get repo
! rm -rf CS4180-DL
! git clone --branch feature/prerak2 https://github.com/prerakmody/CS4180-DL

# Step2 - Get Dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtrainval_11-May-2012.tar --directory /content/CS4180-DL/data/dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtrainval_06-Nov-2007.tar --directory /content/CS4180-DL/data/dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtest_06-Nov-2007.tar --directory /content/CS4180-DL/data/dataset
! mkdir /content/CS4180-DL/data/weights

! wget -P /content/CS4180-DL/data/weights https://pjreddie.com/media/files/yolov2-voc.weights

In [11]:
import os
import sys

%load_ext autoreload
%autoreload 2

## Step3 - Add path to main dir
DIR_MAIN = os.path.abspath('CS4180-DL/') #'../../../CS4180-DL/'
sys.path.append(DIR_MAIN)

## Step4 - Download pruned weights from GDrive
from src.utils import download_gdrive
FILE_ID =  '1dHnUQ8G3GObZSMh9eQ0zUR5wor0ttW3U'
DIR_WEIGHTS = os.path.join(DIR_MAIN, 'data/weights/pruned')
! mkdir {DIR_WEIGHTS}
DEST_NAME_ZIP   = os.path.join(DIR_WEIGHTS, 'weights-prune-me.zip')
DEST_NAME_FILES = DIR_WEIGHTS
download_gdrive(FILE_ID, DEST_NAME_ZIP)
! unzip {DEST_NAME_ZIP} -d {DEST_NAME_FILES}

## Step5 - Generate .txt files for training/validation 
from src.dataloader import setup_VOC
DIR_DATA = os.path.join(DIR_MAIN, 'data/dataset/')
setup_VOC(DIR_DATA)
DIR_DATA_VOC = os.path.join(DIR_DATA, 'VOCdevkit')
! cat {DIR_DATA_VOC}/2007_train.txt {DIR_DATA_VOC}/2007_val.txt {DIR_DATA_VOC}/2012_*.txt > {DIR_DATA_VOC}/voc_train.txt

## Step6 - Check which GPU
! nvidia-smi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 - year :  2012  || image_set :  train
 - year :  2012  || image_set :  val
 - year :  2007  || image_set :  train
 - year :  2007  || image_set :  val
 - year :  2007  || image_set :  test


# Start - Code

In [ ]:
import os
import sys

import torch

dir_main = os.path.abspath('CS4180-DL')
sys.path.append(dir_main)
print (' - In Path : ', sys.path[-1])

USE_GPU = torch.cuda.is_available()
print (' - USE_GPU : ', USE_GPU)

!nvidia-smi

# Start - Predict

In [ ]:
torch.cuda.empty_cache()
# from src.predict import *

if (1):
    DIR_PROJ         = 'CS4180-DL'

if (1):
    MODEL            = ''
    MODEL_CFGFILE    = os.path.join(DIR_PROJ, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
    MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/pruned/yolov2-voc-weight-prune-70.0.weights')
    PASCAL_DIR       = os.path.join(DIR_PROJ, 'data/dataset/VOCdevkit/')
    EVAL_IMAGELIST   = os.path.join(DIR_PROJ, 'data/dataset/VOCdevkit/2007_test.txt')
    EVAL_OUTPUTDIR   = os.path.join(DIR_PROJ, 'eval_data')
    EVAL_PREFIX      = 'iter1_weighspruned_70_'
    EVAL_OUTPUTDIR_PKL = os.path.join(DIR_PROJ, 'eval_results')
    print (' - 0. MODEL : ', MODEL)
    print (' - 0. EVAL_PREFIX : ', EVAL_PREFIX)



valObj = PASCALVOCEval(MODEL, MODEL_CFGFILE, MODEL_WEIGHTFILE, PASCAL_DIR, EVAL_IMAGELIST, EVAL_OUTPUTDIR, EVAL_PREFIX, EVAL_OUTPUTDIR_PKL)
valObj.predict(BATCH_SIZE=32)
# valObj._do_python_eval()

# Start - Training

In [13]:
torch.cuda.empty_cache()
from src.train import YOLOv2Train
from tensorboardcolab import TensorBoardColab

if (1):
    DIR_MAIN         = 'CS4180-DL'
    print (' - 1. DIR_MAIN :  ', DIR_MAIN)
    
if (1):
    PASCAL_DIR   = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/')
    PASCAL_TRAIN = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/voc_train.txt')
    PASCAL_VALID = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/2007_test.txt')
    TRAIN_LOGDIR = os.path.join(DIR_MAIN, 'train_data')
    VAL_LOGDIR   = os.path.join(DIR_MAIN, 'eval_data')
    VAL_OUTPUTDIR_PKL = os.path.join(DIR_MAIN, 'eval_results')
    MODEL_CFG    = os.path.join(DIR_MAIN, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
    # MODEL_WEIGHT = os.path.join(DIR_MAIN, 'data/weights/pruned/yolov2-voc-weight-prune-30.0.weights')
    MODEL_WEIGHT = os.path.join(DIR_MAIN, 'data/dataset/weights/yolov2-voc.weights')
    print (' - 2. MODEL_WEIGHT :  ', MODEL_WEIGHT)
    
if (1):
    VAL_PREFIX   = 'pretrained'
    BATCH_SIZE   = 1;
    print (' - 3. VAL_PREFIX : ', VAL_PREFIX)

if (1):
    try:
        print (' - 4. Logger : ', LOGGER)
    except:
        LOGGER = TensorBoardColab()
        print (' - 4. Logger : ', LOGGER)


if (1):
    trainObj = YOLOv2Train()
    trainObj.train(PASCAL_DIR, PASCAL_TRAIN, PASCAL_VALID, TRAIN_LOGDIR, VAL_LOGDIR, VAL_OUTPUTDIR_PKL, VAL_PREFIX
                   , MODEL_CFG, MODEL_WEIGHT
                   , BATCH_SIZE
                   , LOGGER)

  0%|          | 0/16551 [00:00<?, ?it/s]


 -- init_epoch :  0
 -- max_epochs :  5
 ---------------------------- EPOCH :  0  ---------------------------------- 


  0%|          | 1/16551 [00:00<2:20:53,  1.96it/s]/home/strider/anaconda3/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


 - data (or X) :  torch.Size([1, 3, 416, 416]) torch.float32
 - target (or Y) :  torch.Size([1, 250]) torch.float64
 - Total train points :  16551  || nsamples :  16551
 i : 0 / 16551


/home/strider/anaconda3/lib/python3.5/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
  0%|          | 6/16551 [00:04<4:04:07,  1.13it/s]


KeyboardInterrupt: 